In [149]:
#initialize python and import pandas/numpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.preprocessing import PolynomialFeatures

In [150]:
data = pd.read_excel("data/Master file Jan23.xls")

In [151]:
def clean(data):
    temp_df = data
    # aggregate % adults in original state
    temp_df["Adults_orig"] = temp_df.loc[:, ["Adults_1925_orig", "Adults_2634_orig", "Adults_3554_orig",
                                            "Adults_5564_orig", "Adults_65_orig"]].sum(axis=1)
    temp_df = temp_df.drop(columns = ["Adults_1925_orig", "Adults_2634_orig", "Adults_3554_orig",
                                            "Adults_5564_orig", "Adults_65_orig"])
    # aggregate % adults in destination state
    temp_df["Adults_dest"] = temp_df.loc[:, ["Adults_1925_dest", "Adults_2634_dest", "Adults_3554_dest",
                                            "Adults_5564_dest", "Adults_65_dest"]].sum(axis=1)
    temp_df = temp_df.drop(columns = ["Adults_1925_dest", "Adults_2634_dest", "Adults_3554_dest",
                                            "Adults_5564_dest", "Adults_65_dest"])

    # drop unneccesary columns
    temp_df = temp_df.drop(columns = ["Homic_death_orig","Homic_death_dest","Unemp_tot_orig","Unemp_tot_dest",
                    "StateNAME_orig","StateNAME_dest","outmigsharepercent", "inmigsharepercent",
                    "PDSI_Rank_orig","PDSI_Rank_dest","PDSI_Anomaly_orig", "PDSI_Anomaly_dest",
                    "Urban_rural_orig","Urban_rural_dest", 'MHV_Nom_orig','MGR_Nom_orig','AGR_Nom_orig',
                    'Average_orig', 'MHV_Nom_dest','MGR_Nom_dest','AGR_Nom_dest','Average_dest',
                    'Dem_Share_President_Two_orig', 'Rep_Share_President_Two_orig',
                    'Dem_Share_President_Two_dest', 'Rep_Share_President_Two_dest',
                    'Health_White_orig','Health_Black_orig','Health_Hispanic_orig','Health_Asian_orig',
                    'Health_Alaska_orig','Other_orig', 'Health_White_dest','Health_Black_dest',
                    'Health_Hispanic_dest','Health_Asian_dest','Health_Alaska_dest','Other_dest'])
    
    # rename poorly named columns
    temp_df = temp_df.rename(columns = {"Children_018_orig": "Children_orig", "Children_018_dest": "Children_dest",
                                       "Econ_Free_Sum_dest": "Econ_Freedom_Score_dest",
                                        "Econ_Free_Sum_orig": "Econ_Freedom_Score_orig",
                                       "Reg_Pri_Par_orig": "Price_Parity_orig", "Reg_Pri_Par_dest": "Price_Parity_dest",
                                       "cdd_orig": "cool_deg_day_orig", "cdd_dest": "cool_deg_day_dest",
                                       "hdd_orig": "heat_deg_day_orig", "hdd_dest": "heat_deg_day_dest"})
    

    
    return temp_df

In [152]:
cleaned_data = clean(data)
graph_data=cleaned_data.copy() 

In [153]:
#function to create time-series for a list of dif states and columns
def plot_time_series_data(column_list, state_list, data):
    for column in column_list:
        for state in state_list:
            # Create a sub-dataframe for the current state
            state_data = data[data['StateABRV_orig'] == state]
            
            # Create bar plot for econ var and state
            fig, ax = plt.subplots()
            ax.bar(state_data['Year'], state_data[column])
            ax.set_xlabel('Year')
            ax.set_ylabel(column)
            ax.set_title(f'{column} in {state}')
            
            # Show the plot
            plt.show()

def fit_to_polynomial(data, years, column, state, degree=2):
    #create state data frame
    state_df = data[data['StateABRV_orig']==state]
    
    #drop na values
    no_nulls_df = state_df.dropna(subset=[column, 'Year'])
    no_nulls_df = no_nulls_df[pd.notnull(no_nulls_df[column])]
    
    #designate x and y
    x = np.array(no_nulls_df['Year'])
    y = np.array(no_nulls_df[column])
    
    #fit polynomial models up to degree 5
    coeffs = np.polyfit(x, y, degree)
    model = np.poly1d(coeffs)
    
#     # plot the scatterplot
#     plt.scatter(x, y)

#     # plot the fitted polynomial line
#     plt.plot(x, model(x), color='green')
    
#     plt.show()
    
    # predict the values for the given years
    predicted_values = model(years)
    
    return predicted_values

In [154]:
#make a list of all the columns we want to get estimates for:
null_columns = graph_data[graph_data.columns[graph_data.isna().any()]]
null_columns_list =sorted(null_columns.columns.values.tolist())

#edit list to have only orig values
imputation_columns=['Average_Gross_Rent_adj_orig']
#,'Coll_Educ_orig', 'Health_Adults_orig','Homicide_Rate_orig','Inc_Corp_Tax_orig',
#                  'Median_Gross_Rent_adj_orig','Median_House_Value_adj_orig', 'PDSI_Value_orig'

In [155]:
# make dictionary w/ columns and where their nulls are
null_dict = {}
for col in imputation_columns:
    null_indices = graph_data[graph_data[col].isnull()].index.tolist()
    null_dict[col] = null_indices


In [156]:
#code to run estimations for alll of the variables

years=[2013,2014]
for col in imputation_columns:
    estimated_values = fit_to_polynomial(data, [2013, 2014], col, 'AL', 3)
    print('column: ', col)
    print('estimated values: ', estimated_values)

column:  Average_Gross_Rent_adj_orig
estimated values:  [726.0932374  726.06293344]


In [157]:
graph_data[(graph_data["Year"] == 2009) & (graph_data["StateCODE_orig"] == 2)]

,Year,StateCODE_orig,StateCODE_dest,Mig_Estimate,outmigshare,inmigshare,Distance,StateABRV_orig,StateABRV_dest,Pop_orig,...,Price_Parity_dest,Econ_Freedom_Score_dest,Homeown_Perc_dest,Median_HHI_dest,Median_HHI_adj_dest,Coll_Educ_dest,heat_deg_day_dest,cool_deg_day_dest,Adults_orig,Adults_dest
22099,2009,2,1,154.0,0.000220,0.000032,926,AK,AL,698895,...,88.685,5.84,74.1,39980,46905,22.0,2792,1824,0.71,0.743
22100,2009,2,3,818.0,0.001170,0.000129,2584,AK,AZ,698895,...,103.615,6.04,68.9,45739,53662,25.6,1894,3086,0.71,0.724
22101,2009,2,4,6036.0,0.008636,0.002084,840,AK,AR,698895,...,87.468,5.75,68.5,36538,42867,18.9,3519,1512,0.71,0.740
22102,2009,2,5,334.0,0.000478,0.000009,609,AK,CA,698895,...,109.211,4.11,57.0,56134,65858,29.9,2879,967,0.71,0.730
22103,2009,2,6,56135.0,0.080320,0.011290,1106,AK,CO,698895,...,102.001,6.25,68.4,55930,65618,35.9,7354,189,0.71,0.740
22104,2009,2,7,631.0,0.000903,0.000177,352,AK,CT,698895,...,109.135,6.47,70.5,64851,76085,35.6,6106,443,0.71,0.753
22105,2009,2,8,87.0,0.000124,0.000098,1388,AK,DE,698895,...,101.927,5.67,76.5,52114,61141,28.7,4617,978,0.71,0.754
22106,2009,2,9,3846.0,0.005503,0.000206,1282,AK,FL,698895,...,100.873,6.50,70.9,45631,53535,25.3,705,3576,0.71,0.767
22107,2009,2,10,NaN,NaN,NaN,1314,AK,GA,698895,...,92.300,6.20,67.4,43340,50848,27.5,2981,1638,0.71,0.717
22108,2009,2,11,1142.0,0.001634,0.000848,1079,AK,HI,698895,...,112.207,5.10,59.5,55649,65289,29.6,0,4431,0.71,0.756


In [158]:
def impute(data, null_dict):
    # go thru dictionary, imputing vals at each index per column
    # return a df with the imputed vals
    states = data["StateABRV_orig"].unique().tolist()
    imputed = data.copy()

    for col in null_dict:
        for state in states:
            idx = null_dict[col]
            years = graph_data.loc[idx]["Year"].unique().tolist()
            est_vals = fit_to_polynomial(data, years, col, state, 3)
            year_and_vals = dict(zip(years, est_vals))
            # i have the estimated values, but each year occurs multiple times
            # not sure how to insert the values in the right spots
            # should need to impute 49 times for each year  
            for year_and_val in year_and_vals:
                imputed.at[year_and_val, col] = year_and_vals[year_and_val]
    
    return imputed
            

In [160]:
imputed = impute(cleaned_data, null_dict)

In [162]:
imputed

,Year,StateCODE_orig,StateCODE_dest,Mig_Estimate,outmigshare,inmigshare,Distance,StateABRV_orig,StateABRV_dest,Pop_orig,...,Price_Parity_dest,Econ_Freedom_Score_dest,Homeown_Perc_dest,Median_HHI_dest,Median_HHI_adj_dest,Coll_Educ_dest,heat_deg_day_dest,cool_deg_day_dest,Adults_orig,Adults_dest
0,2000,1,2,608.0,0.000137,0.000968,3459,AL,AK,4452173,...,NaN,3.65,66.4,52847,77275,NaN,10392,4,0.000,0.000
1,2000,1,3,574.0,0.000129,0.000111,1485,AL,AZ,4452173,...,NaN,6.67,68.0,39783,58172,NaN,1973,3155,0.000,0.000
2,2000,1,4,3168.0,0.000712,0.001183,347,AL,AR,4452173,...,NaN,6.40,68.9,29697,43424,NaN,3532,1901,0.000,0.000
3,2000,1,5,3025.0,0.000679,0.000089,1883,AL,CA,4452173,...,NaN,5.30,57.1,46816,68456,NaN,2830,831,0.000,0.000
4,2000,1,6,321.0,0.000072,0.000074,1143,AL,CO,4452173,...,NaN,7.65,68.3,48240,70538,NaN,6874,419,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41645,2016,50,45,0.0,0.000000,0.000000,1699,WY,VT,584215,...,99.491,5.15,71.3,60837,63660,36.4,7452,381,0.746,0.797
41646,2016,50,46,48.0,0.000082,0.000006,1528,WY,VA,584215,...,103.022,7.31,66.3,66451,69535,38.1,3958,1548,0.746,0.763
41647,2016,50,47,866.0,0.001482,0.000119,734,WY,WA,584215,...,104.816,6.18,61.6,70310,73573,35.1,4847,347,0.746,0.764
41648,2016,50,48,0.0,0.000000,0.000000,1388,WY,WV,584215,...,90.482,4.42,74.8,44354,46412,20.8,4716,1148,0.746,0.783


In [15]:
STILL TO DO:
    - edit imputations so that it picks the best degree based on the set of points
    - change from a list of columns with missing values to a dictionary
        - use this dictionary to store both the column names AND the years we need to impute
    - get these imputed values into the dataframe (well into a copy of the data frame)
    -cry tears of joy if u get it to work



SyntaxError: invalid syntax (525025169.py, line 1)